In [7]:
import csv
import datetime
import numpy as np
import os
import pandas as pd
import random
import scipy
import xarray as xr

import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [5]:
dirs = {'AM2.5_tc_ctrl': '/tigress/wenchang/analysis/TC/AM2.5/CTL1990s_tigercpu_intelmpi_18_540PE/model_out/POSTP',
        'AM2.5_tc_ktc2': '/tigress/wenchang/analysis/TC/AM2.5ktc2/CTL1990s_killtc13-13-15_tigercpu_intelmpi_18_540PE/modelout/POSTP',
        'HIRAM_tc_ctrl': '/tigress/wenchang/analysis/TC/HIRAM/CTL1990s_v201910_tigercpu_intelmpi_18_540PE/model_out/POSTP',
        'HIRAM_tc_ktc2': '/tigress/wenchang/analysis/TC/HIRAMktc2/CTL1990s_v201910_killtc13-13-15_tigercpu_intelmpi_18_540PE/modelout/POSTP'}

In [9]:
def print_varnames(dirs, atmos_only=True):
    ''' 
    Method to write AM output variables and associated metadata to a Markdown (.md) file. 
    'atmos_only' is a boolean to specify if only atmosphere-specific output will be evaluated.
    '''

    # Create list of output file types
    output_types = set([f.split('.')[1] 
                        for k, v in dirs.items() 
                        for f in os.listdir(v) 
                        if (f.split('.')[-1] == 'nc') & ('AM2.5' in k)])
    # Only select atmospheric output if boolean True
    if atmos_only:
        output_types = [o for o in output_types if 'atmos' in o]
        
    # Initialize container for rows
    rows = []
    # Iterate through each output type to inspect contained variables
    for k, v in dirs.items():
        for output_type in output_types:
            # Get a sample file matching the iterand output type
            fname = [os.path.join(v, f) for f in os.listdir(v) if output_type in f][0]
            # Pull dataset
            ds = xr.open_dataset(fname)
            for varname in ds.data_vars.keys():
                try:
                    units = ds[varname].attrs['units']
                except:
                    units = ''
                row_out = '{0},;{1},;{2},;{3},;{4}'.format(k.split('_')[0], output_type, varname, ds[varname].attrs['long_name'], units)
                rows.append(row_out.split(',;'))
    # Define a header        
    header = ['Model', 'Output file type', 'Variable', 'Variable name', 'Units']
    # Write to file
    fname = '/projects/GEOCLIM/grios/backup/varnames.csv' if not atmos_only else '/projects/GEOCLIM/grios/backup/varnames_atmos.csv'
    with open(fname, 'wt') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for row in rows:
            writer.writerow(row)
    
    fname_md = fname.split('.')[0] + '.md'
    pd.read_csv(fname).to_markdown(fname_md, tablefmt='github', index=False)

In [11]:
print_varnames(dirs, atmos_only=False)